# Hopping leg

In [23]:
from os import path
import mujoco
import mujoco_viewer
import numpy as np
from matplotlib import pyplot as plt

In [24]:
def set_torque_servo(actuator_no, flag):
    if (flag == 0):
        model.actuator_gainprm[actuator_no, 0] = 0
    else:
        model.actuator_gainprm[actuator_no, 0] = 1


def controller(model, mjdata, q_A: float, q_B: float):
    ID_JOINT_A = 0
    ID_JOINT_B = 1
    TAU_ID_A = 0
    TAU_ID_B = 1
    KP = 0
    KD = 0

    set_torque_servo(0, 1)
    mjdata.ctrl[TAU_ID_A] = KP*(q_A-mjdata.qpos[ID_JOINT_A]) + KD*(0-mjdata.qvel[ID_JOINT_A])
    mjdata.ctrl[TAU_ID_B] = KP*(q_B-mjdata.qpos[ID_JOINT_B]) + KD*(0-mjdata.qvel[ID_JOINT_B])

SIMEND = 10
TIMESTEP = 0.005
STEP_NUM = int(SIMEND/TIMESTEP)
MODEL_NAME = 'open_chain_leg_scene.xml'

ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, '')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

# set reference
i = 0
time = 0
tt = np.linspace(0, SIMEND, STEP_NUM)
freq = 1
ref = [np.pi/12 * np.sin(freq * tt) + np.pi/4,
       -np.pi/6 * np.sin(freq * tt) - np.pi/2]

x_ee_all = []
z_ee_all = []
theta1_all = []
theta2_all = []

# set initial angles
data.qpos[1] = 0.78  # set position
data.qpos[2] = -1.57  # set position
mujoco.mj_forward(model, data)

In [25]:
viewer = mujoco_viewer.MujocoViewer(model, data,
                                    title = "open_chain_leg",
                                    width = 1920,
                                    height = 1080)

for i in range(STEP_NUM):
    if viewer.is_alive:
        # time_prev = data.time
        # while (data.time - time_prev < 1.0/60.0):
        mujoco.set_mjcb_control(controller(model, data, ref[0][i], ref[1][i]))
        mujoco.mj_step(model, data)
        viewer.render()
    else:
        print("pam pam", i)
        break

viewer.close()